<a href="https://colab.research.google.com/github/bf319/Scaling_MPNNs/blob/main/final_version_3_try_to_use_pmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q git+https://github.com/deepmind/dm-haiku
%pip install -q jraph
%pip install -q git+https://github.com/deepmind/jaxline
%pip install -q ogb
%pip install -q dgl
%pip install -q optax
%pip install -q metis

# Question -- when installing jraph with pip, jraph.experimental is not found so this is 
# the easiest way I have found to work with it.
!wget https://raw.githubusercontent.com/deepmind/jraph/master/jraph/experimental/sharded_graphnet.py

--2022-03-21 21:18:15--  https://raw.githubusercontent.com/deepmind/jraph/master/jraph/experimental/sharded_graphnet.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22444 (22K) [text/plain]
Saving to: ‘sharded_graphnet.py.1’

sharded_graphnet.py 100%[===================>]  21.92K  --.-KB/s    in 0.001s  

2022-03-21 21:18:15 (29.5 MB/s) - ‘sharded_graphnet.py.1’ saved [22444/22444]



In [2]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

num_devices = jax.local_device_count()
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [3]:
from ogb.nodeproppred import Evaluator
from ogb.nodeproppred import DglNodePropPredDataset

dataset = DglNodePropPredDataset(name = "ogbn-proteins")
split_idx = dataset.get_idx_split()
evaluator = Evaluator(name = 'ogbn-proteins')

Using backend: pytorch


In [48]:
import jax.numpy as jnp

# There is only one graph in Node Property Prediction datasets
ogbn_proteins_main_graph, ogbn_proteins_main_labels = dataset[0]

'''
  OGBN-Proteins
    #Nodes = 132,534
    #Edges = 39,561,252
    #Tasks = 112
    #Split Type = Species
    #Task Type = Binary classification
    #Metric = ROC-AUC

    Task:
      The task is to predict the presence of protein functions in a multi-label binary classification setup,
      where there are 112 kinds of labels to predict in total. 
      The performance is measured by the average of ROC-AUC scores across the 112 tasks.

    #Others:
      **undirected**
      **weighted**
      **typed (according to species)**

  (1) Nodes represent proteins
    (1.1) The proteins come from 8 species
      len(set(graph.ndata['species'].reshape(-1).tolist())) == 8
    (1.2) Each node has one feature associated with it (its species)
      graph.ndata['species'].shape == (#nodes, 1)
  
  (2) Edges indicate different types of biologically meaningful associations between proteins
    (2.1) All edges come with 8-dimensional features
      graph.edata['feat'].shape == (2 * #edges, 8)

'''
# Get split labels
train_label = dataset.labels[split_idx['train']]  # (86619, 112) -- binary values (presence of protein functions)
valid_label = dataset.labels[split_idx['valid']]  # (21236, 112) -- binary values (presence of protein functions)
test_label = dataset.labels[split_idx['test']]    # (24679, 112) -- binary values (presence of protein functions)

# Create masks
train_mask = jnp.zeros((ogbn_proteins_main_graph.num_nodes(), 1)).at[jnp.array(split_idx['train'])].set(1)
valid_mask = jnp.zeros((ogbn_proteins_main_graph.num_nodes(), 1)).at[jnp.array(split_idx['valid'])].set(1)
test_mask = jnp.zeros((ogbn_proteins_main_graph.num_nodes(), 1)).at[jnp.array(split_idx['test'])].set(1)

In [5]:
import jraph

# From https://colab.research.google.com/github/deepmind/educational/blob/master/colabs/summer_schools/intro_to_graph_nets_tutorial_with_jraph.ipynb#scrollTo=7vEmAsr5bKN8
def _nearest_bigger_power_of_two(x: int) -> int:
  """Computes the nearest power of two greater than x for padding."""
  y = 2
  while y < x:
    y *= 2
  return y

def pad_graph_to_nearest_power_of_two(
    graphs_tuple: jraph.GraphsTuple) -> jraph.GraphsTuple:
  """Pads a batched `GraphsTuple` to the nearest power of two.
  For example, if a `GraphsTuple` has 7 nodes, 5 edges and 3 graphs, this method
  would pad the `GraphsTuple` nodes and edges:
    7 nodes --> 8 nodes (2^3)
    5 edges --> 8 edges (2^3)
  And since padding is accomplished using `jraph.pad_with_graphs`, an extra
  graph and node is added:
    8 nodes --> 9 nodes
    3 graphs --> 4 graphs
  Args:
    graphs_tuple: a batched `GraphsTuple` (can be batch size 1).
  Returns:
    A graphs_tuple batched to the nearest power of two.
  """
  # Add 1 since we need at least one padding node for pad_with_graphs.
  pad_nodes_to = _nearest_bigger_power_of_two(jnp.sum(graphs_tuple.n_node)) + 1
  pad_edges_to = _nearest_bigger_power_of_two(jnp.sum(graphs_tuple.n_edge))
  # Add 1 since we need at least one padding graph for pad_with_graphs.
  # We do not pad to nearest power of two because the batch size is fixed.
  pad_graphs_to = graphs_tuple.n_node.shape[0] + 1
  return jraph.pad_with_graphs(graphs_tuple, pad_nodes_to, pad_edges_to,
                               pad_graphs_to)

In [9]:
import numpy as np
import torch
import jraph

import sharded_graphnet

def dgl_graph_to_jraph(node_ids):
  # First add back the node and edge features
  dgl_graph_with_features = dgl.node_subgraph(ogbn_proteins_main_graph, node_ids)

  node_features = jnp.array(dgl_graph_with_features.ndata['species'])
  senders = jnp.array(dgl_graph_with_features.edges()[0])
  receivers = jnp.array(dgl_graph_with_features.edges()[1])

  # Edges -- here we should include the 8-dimensional edge features
  edges = jnp.array(dgl_graph_with_features.edata['feat'])

  n_node = jnp.array([dgl_graph_with_features.num_nodes()])
  n_edge = jnp.array([dgl_graph_with_features.num_edges()])

  in_tuple = jraph.GraphsTuple(
            nodes = node_features.astype(np.float32),
            senders = senders.astype(np.int32), 
            receivers = receivers.astype(np.int32),
            edges = edges.astype(np.float32),  
            n_node = n_node, 
            n_edge = n_edge,
            globals = None  # No global features
          )
  
  # Question -- do we need padding yet? I have found it being used in most
  # of the code available online, so I figured this might be useful in the future.
  # However, I am not really sure what type of padding we would need.

  in_tuple = pad_graph_to_nearest_power_of_two(in_tuple)
  
  return sharded_graphnet.graphs_tuple_to_broadcasted_sharded_graphs_tuple(
      in_tuple,
      num_shards = num_devices
      )
  
def get_labels_for_subgraph(node_ids):
  return jnp.array(ogbn_proteins_main_labels.index_select(0, node_ids))

In [10]:
import dgl

'''
  Generate graph partition using metis, with balanced number of edges in each partition.
  Note: 
    The subgraphs do not contain the node/edge data in the input graph (https://docs.dgl.ai/generated/dgl.metis_partition.html)
'''
num_partitions = 400  ## TODO: Find some way to decrease this to something reasonable (< 50)

# reshuffle == False, so my understandinng is that
# partition.ndata['_ID'] uses the same ids for the initial nodes
# therefore, the initial train / valid / test masks should still work.
dgl_graph_metis_partition = dgl.metis_partition(ogbn_proteins_main_graph, num_partitions, balance_edges = True)

Convert a graph into a bidirected graph: 2.647 seconds
Construct multi-constraint weights: 0.003 seconds
Metis partitioning: 46.809 seconds
Split the graph: 0.680 seconds
Construct subgraphs: 0.097 seconds


In [11]:
# Convert graphs to Jraph GraphsTuple
processed_graphs = {}

for idx in range(num_partitions):
  node_ids = dgl_graph_metis_partition[idx].ndata['_ID']

  graph = dgl_graph_to_jraph(node_ids)
  labels = get_labels_for_subgraph(node_ids)

  processed_graphs[f'partition_{idx}'] = {
      'graph': graph, 
      'labels': labels,
      'train_mask': train_mask.at[jnp.array(node_ids)].get(),
      'valid_mask': valid_mask.at[jnp.array(node_ids)].get(),
      'test_mask': test_mask.at[jnp.array(node_ids)].get()
      }

In [15]:
import haiku as hk
import jax
import optax

from typing import Sequence

# See https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py for alternative to using GraphMapFeatures
# From https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py

hidden_dimension = 64
num_message_passing_steps = 3

@jraph.concatenated_args
def node_update_fn(feats: jnp.ndarray) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.Sequential([hk.nets.MLP(output_sizes = [hidden_dimension, hidden_dimension]), jax.nn.relu, hk.LayerNorm(axis = -1, create_scale = False, create_offset = False)])
  return net(feats)

@jraph.concatenated_args
def edge_update_fn(feats: jnp.ndarray) -> jnp.ndarray:
  """Edge update function for graph net."""
  net = hk.Sequential([hk.nets.MLP(output_sizes = [hidden_dimension, hidden_dimension]), jax.nn.relu, hk.LayerNorm(axis = -1, create_scale = False, create_offset = False)])
  return net(feats)

def node_decoder_fn(feats: jnp.ndarray) -> jnp.ndarray:
  ## TODO: Check if this is correct
  net = hk.Sequential([hk.nets.MLP(output_sizes = [112], activate_final = False), jax.nn.sigmoid])
  return net(feats)

# Error when I try to use these annotations: https://pastebin.com/GeRddqbi
# @hk.transform
# @jax.pmap
@hk.without_apply_rng
@hk.transform
def network_definition(graph):
  """Defines a graph neural network.
  Args:
    graph: Graphstuple the network processes.
  Returns:
    Decoded nodes.
  """

  # Question -- I cannot use this embedder with a ShardedGraphNet
  # but I would think this should not be much of a problem anyway because the
  # embedder is just adding one more Linear layer right?

  # embedder = jraph.GraphMapFeatures(
  #     embed_node_fn=hk.Linear(hidden_dimension),
  #     embed_edge_fn=hk.Linear(hidden_dimension),
  #     )
  # graph = embedder(graph)
  # TODO: Replace embedded with Linear layer
  
  # Error when I do not use pmap https://pastebin.com/cH9sK0wr
  sharded_gn = sharded_graphnet.ShardedEdgesGraphNetwork(
      update_node_fn = node_update_fn,
      update_edge_fn = edge_update_fn,
      num_shards = num_devices
      )

  for _ in range(num_message_passing_steps):
    graph = sharded_gn(graph)

  graph = graph._replace(
      nodes = hk.Linear(112)(graph.nodes)
  )
  return graph.nodes

In [16]:
def bcast_local_devices(value):
    """Broadcasts an object to all local devices."""
    devices = jax.local_devices()

    def _replicate(x):
      """Replicate an object on each device."""
      x = jnp.array(x)
      return jax.device_put_sharded(len(devices) * [x], devices)

    return jax.tree_util.tree_map(_replicate, value)

In [55]:
import functools
import haiku as hk

from random import randint

# Based on https://github.com/deepmind/optax/blob/master/optax/_src/loss.py#L116#L139
def custom_binary_cross_entropy(logits, labels, mask):
  # logits.shape == mask.shape > labels.shape
  log_p = jax.nn.log_sigmoid(logits)

  # log(1 - sigmoid(x)) = log_sigmoid(-x), the latter more numerically stable
  log_not_p = jax.nn.log_sigmoid(-logits)
  return -labels * log_p - (1. - labels) * log_not_p


# Try to follow this tutorial https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py
def compute_loss(params, graph, label, mask):
  predictions = network_definition.apply(params, graph)
  label = jnp.zeros_like(predictions)

  # predictions_after_masked_nodes_are_removed = predictions.at[jraph.get_node_padding_mask(graph)].get() # Shape == label.shape
  predictions_after_masked_nodes_are_removed = predictions

  # Question: Is node masking implemented properly?
  # Masking as in both
  # (1) Masking of nodes added from padding
  # (2) Masking of nodes from the train / eval / test sets
  loss = optax.sigmoid_binary_cross_entropy(predictions_after_masked_nodes_are_removed, label)
  loss = loss * mask
  loss = loss.mean()
  return loss

def train(num_training_steps):
  replicated_params = jax.pmap(network_definition.init, axis_name = 'i')(bcast_local_devices(jax.random.PRNGKey(42)), processed_graphs['partition_0']['graph'])

  opt_init, opt_update = optax.adam(learning_rate = 0.1)  
  replicated_opt_state = bcast_local_devices(opt_init(replicated_params))

  @functools.partial(jax.pmap, axis_name='i')
  def update(params, opt_state, graph, targets, mask):
    # Compute the gradients on the given minibatch (individually on each device).
    loss, grads = jax.value_and_grad(compute_loss)(params, graph, targets, mask)

    # # Combine the gradient across all devices (by taking their mean).
    grads = jax.lax.pmean(grads, axis_name='i')

    # # Also combine the loss. Unnecessary for the update, but useful for logging.
    loss = jax.lax.pmean(loss, axis_name='i')

    updates, opt_state = opt_update(grads, opt_state)
    return optax.apply_updates(params, updates), opt_state, loss

  # Train
  for idx in range(num_training_steps):
    random_partition_idx = randint(0, num_partitions - 1)
    random_partition = processed_graphs[f'partition_{random_partition_idx}']

    graph = random_partition['graph']
    labels = bcast_local_devices(random_partition['labels'])
    mask = bcast_local_devices(random_partition['train_mask'])

    params, opt_state, loss = update(
        replicated_params, 
        replicated_opt_state, 
        graph, 
        labels,
        mask
        )
    print('Loss training:', loss)

    # Write params and opt_state

  return params

def evaluate(params, num_graphs_eval):
  # Evaluate
  accumulated_loss = 0.0
  accumulated_roc = 0

  for idx in range(num_graphs_eval):
    random_partition_idx = randint(0, num_partitions - 1)
    random_partition = processed_graphs[f'partition_{random_partition_idx}']

    graph = random_partition['graph']
    labels = random_partition['labels']
    mask = random_partition['test_mask']

    (loss, roc) = evaluate_graph(params, graph, labels, mask)

    accumulated_loss += loss
    accumulated_roc += roc

  print(f'Average test loss: {accumulated_loss / num_graphs_eval} | Average ROC: {accumulated_roc / num_graphs_eval}')

def evaluate_graph(params, graph, label, mask):
  # compute_loss_fn = jax.jit(functools.partial(compute_loss))
  compute_loss_fn = functools.partial(compute_loss)

  decoded_nodes = network_definition.apply(params, graph)
  
  ## Question: Predictions should be rounded here right?
  predictions = jax.lax.round(decoded_nodes)
  predictions_after_masked_nodes_are_removed = predictions.at[jraph.get_node_padding_mask(graph)].get() # Shape == label.shape

  loss = compute_loss_fn(params, graph, label, mask)
  roc = evaluator.eval({"y_true": np.array(label), "y_pred": np.array(predictions_after_masked_nodes_are_removed)})['rocauc']

  print(f'Test loss: {loss} | ROC: {roc}')
  return (loss, roc)

final_params = train(num_training_steps = 10)
evaluate(final_params, 5)

TypeError: ignored

In [ ]:
# import functools
# import haiku as hk

# from random import randint

# # Try to follow this tutorial https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py
# def compute_loss(params, graph, label, mask):
#   net = hk.without_apply_rng(hk.transform(network_definition))

#   predictions = jax.pmap(net.apply)(params, graph)
#   # predictions = net.apply(params, graph)
#   predictions_after_masked_nodes_are_removed = predictions.at[jraph.get_node_padding_mask(graph)].get() # Shape == label.shape
  
#   # Question: Is node masking implemented properly?
#   # Masking as in both
#   # (1) Masking of nodes added from padding
#   # (2) Masking of nodes from the train / eval / test sets
#   loss = optax.sigmoid_binary_cross_entropy(predictions_after_masked_nodes_are_removed, label)
#   loss = loss * mask
#   loss = loss.mean()
#   return loss

# def train(num_training_steps):
#   # Transform the function (MPNN) into a pure function (with no side effects) so that it can be used with jax
#   # net = hk.without_apply_rng(hk.transform(network_definition))
#   # net = hk.without_apply_rng(hk.transform(network_definition))

#   # params = net.init(jax.random.PRNGKey(42), processed_graphs['partition_0']['graph'])

#   net = hk.without_apply_rng(hk.transform(network_definition))
#   p_init = jax.pmap(net.init, axis_name = 'i')

#   params = p_init(bcast_local_devices(jax.random.PRNGKey(42)), processed_graphs['partition_0']['graph'])

#   opt_init, opt_update = optax.adam(learning_rate = 0.1)  
#   opt_state = opt_init(params)

#   # replicated_params = jax.tree_map(lambda x: jnp.array([x] * num_devices), params)
#   # replicated_opt_state = jax.tree_map(lambda x: jnp.array([x] * num_devices), opt_state)

#   replicated_params = bcast_local_devices(params)
#   replicated_opt_state = bcast_local_devices(opt_state)

#   # @functools.partial(jax.pmap, axis_name='i')
#   def update(params, opt_state, graph, targets, mask):
#     # Compute the gradients on the given minibatch (individually on each device).
#     loss, grads = jax.pmap(jax.value_and_grad(compute_loss)(params, graph, targets, mask))

#     # # Combine the gradient across all devices (by taking their mean).
#     # grads = jax.lax.pmean(grads, axis_name='num_devices')

#     # # Also combine the loss. Unnecessary for the update, but useful for logging.
#     # loss = jax.lax.pmean(loss, axis_name='num_devices')

#     updates, opt_state = opt_update(grads, opt_state)
#     return optax.apply_updates(params, updates), opt_state, loss

#   # Train
#   for idx in range(num_training_steps):
#     random_partition_idx = randint(0, num_partitions - 1)
#     random_partition = processed_graphs[f'partition_{random_partition_idx}']

#     graph = random_partition['graph']
#     labels = random_partition['labels']
#     mask = random_partition['train_mask']

#     labels = bcast_local_devices(labels)
#     mask = bcast_local_devices(mask)

#     params, opt_state, loss = update(
#         replicated_params, 
#         replicated_opt_state, 
#         graph, 
#         labels,
#         mask
#         )
#     print('Loss training:', loss)

#     # Write params and opt_state

#   return params

# def evaluate(params, num_graphs_eval):
#   # Evaluate
#   accumulated_loss = 0.0
#   accumulated_roc = 0

#   for idx in range(num_graphs_eval):
#     random_partition_idx = randint(0, num_partitions - 1)
#     random_partition = processed_graphs[f'partition_{random_partition_idx}']

#     graph = random_partition['graph']
#     labels = random_partition['labels']
#     mask = random_partition['test_mask']

#     (loss, roc) = evaluate_graph(params, graph, labels, mask)

#     accumulated_loss += loss
#     accumulated_roc += roc

#   print(f'Average test loss: {accumulated_loss / num_graphs_eval} | Average ROC: {accumulated_roc / num_graphs_eval}')

# def evaluate_graph(params, graph, label, mask):
#   # compute_loss_fn = jax.jit(functools.partial(compute_loss))
#   compute_loss_fn = functools.partial(compute_loss)

#   net = hk.without_apply_rng(hk.transform(network_definition))
#   decoded_nodes = net.apply(params, graph) # Shape == label.shape
  
#   ## Question: Predictions should be rounded here right?
#   predictions = jax.lax.round(decoded_nodes)
#   predictions_after_masked_nodes_are_removed = predictions.at[jraph.get_node_padding_mask(graph)].get() # Shape == label.shape

#   loss = compute_loss_fn(params, graph, label, mask)
#   roc = evaluator.eval({"y_true": np.array(label), "y_pred": np.array(predictions_after_masked_nodes_are_removed)})['rocauc']

#   print(f'Test loss: {loss} | ROC: {roc}')
#   return (loss, roc)

# final_params = train(num_training_steps = 10)
# evaluate(final_params, 5)

In [ ]:
def evaluate_on_full_sets(params):
  net = hk.without_apply_rng(hk.transform(network_definition))

  final_predictions = {}

  for i in range(num_partitions):
    node_ids = dgl_graph_metis_partition[i].ndata['_ID']

    partition = processed_graphs[f'partition_{i}']
    decoded_nodes = net.apply(params, partition['graph'])
    predictions = jax.lax.round(decoded_nodes)
    predictions_after_masked_nodes_are_removed = predictions.at[jraph.get_node_padding_mask(partition['graph'])].get() # Shape == label.shape
    predictions_after_masked_nodes_are_removed = np.array(predictions_after_masked_nodes_are_removed)

    for index, node_id in enumerate(node_ids):
      final_predictions[node_id] = predictions_after_masked_nodes_are_removed[index]

    if (i + 1) % 10 == 0:
      print(f'Evaluated {i + 1} / {num_partitions} subgraphs...')

  # Sort the final predictions based on the node ids
  predictions_in_order = dict(sorted(final_predictions.items()))

  # Convert the values to a list to be able to slice based on the ids of the 
  # nodes in the test set
  predictions_in_order = list(predictions_in_order.values())

  final_roc_train = evaluator.eval({
      "y_true": np.array(train_label), 
      "y_pred": np.array([predictions_in_order[x] for x in split_idx['train']])
      })['rocauc']

  final_roc_valid = evaluator.eval({
      "y_true": np.array(valid_label), 
      "y_pred": np.array([predictions_in_order[x] for x in split_idx['valid']])
      })['rocauc']

  final_roc_test = evaluator.eval({
      "y_true": np.array(test_label), 
      "y_pred": np.array([predictions_in_order[x] for x in split_idx['test']])
      })['rocauc']

  print()
  print(f'Final ROC on the train set {final_roc_train}')
  print(f'Final ROC on the validation set {final_roc_valid}')
  print(f'Final ROC on the test set {final_roc_test}')

evaluate_on_full_sets(final_params)

In [ ]:
# Question
# I am not really sure I understand what sharding the graph actually means which is why I wanted to confirm this.
# From what I understand, with ShardedEdgesGraphNetworks sharding the graph means replicating the node / node features
# on all devices and splitting the edges of the graphs across all devices. This would mean that on each training step
# I only use a single graph right?

# This is in contrast with using a batch of 8 graphs on each training step, then passing one graph to each device.
# (something similar to this https://github.com/deepmind/jraph/issues/10)